In [18]:
import requests
import pandas as pd
import json
import re

In [19]:
password = pd.read_csv("passwords.csv")
password_nps = password["password"][0]

In [25]:
#pull API using API key
alerts_url = "https://developer.nps.gov/api/v1/alerts?limit=1000&api_key=" + password_nps

response_API = requests.get(alerts_url)
#print(response_API.status_code)
data = response_API.text
alerts_json = json.loads(data)

In [ ]:
data_list_of_dicts = alerts_json['data']
alerts_df = pd.DataFrame(data_list_of_dicts)
#alerts_df.to_csv('alerts.csv', index=False)

In [29]:
road_labels = ['road', 'drive', 'bypass', 'highway', 'parking lot']
closed_labels = ['closed', 'closure', 'closures']

road_pattern = "|".join(map(re.escape, road_labels))
closed_pattern = "|".join(map(re.escape, closed_labels))

road_label_matches = alerts_df['description'].str.contains(road_pattern, case=False, na=False)
closure_label_matches = alerts_df['description'].str.contains(closed_pattern, case=False, na=False)

alerts_df["Actual_Closure"] = (road_label_matches & closure_label_matches).map({True: 'yes', False: 'no'})

alerts_df.to_csv('Alerts_with_closures.csv', index=False)